<a href="https://colab.research.google.com/github/123nishant/Google-Colab/blob/master/Fast_Text_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will use fastText Library to perform text classification on the dbpedie data which can be downloaded from 
https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz


fastText is a library for learning of "Word Embeddings" and "Text Classification" created by FB AI Research Lab. 

This model allows to create an unsupervised learning or supervised learning algorithm for obtaining vector representations for words.

FB makes available pretrained models for 294 languages

This notebook uses an older version of fastText


In [2]:
!pip install fasttext==0.9.2

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3018329 sha256=ea451c9fbf6bfa2f3b698d764f15041dddf8e730dae93bc0e25d89274548df83
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [3]:
import pandas as pd

In [4]:
# downloading the data

!wget -P DATAPATH https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz

# untaring the required file 
!tar -xavf DATAPATH/dbpedia_csv.tar.gz -C DATAPATH

# sneak peek in the folder structure 
!ls -lah DATAPATH

--2020-09-28 12:24:15--  https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/srhrshr/torchDatasets/raw/master/dbpedia_csv.tar.gz [following]
--2020-09-28 12:24:16--  https://github.com/srhrshr/torchDatasets/raw/master/dbpedia_csv.tar.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/srhrshr/torchDatasets/master/dbpedia_csv.tar.gz [following]
--2020-09-28 12:24:16--  https://raw.githubusercontent.com/srhrshr/torchDatasets/master/dbpedia_csv.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connecte

In [5]:
data_path = 'DATAPATH'

# loading train data
train_file = data_path + '/dbpedia_csv/train.csv'
df = pd.read_csv(train_file, header = None, names = ['class', 'name', 'description'])

# Loading the test data
test_file = data_path + '/dbpedia_csv/test.csv'
df_test = pd.read_csv(test_file, header = None, names = ['class', 'name', 'description'])

# Data we have
print("Train: {} Test: {}".format(df.shape, df_test.shape))

Train: (560000, 3) Test: (70000, 3)


In [6]:
# Since we have no clue about the classes lets build one
# Mapping from class number to class name
class_dict = {
    1:'Company',
    2:'EducationalInstitution',
    3:'Artist',
    4:'Athlete',
    5:'OfficeHolder',
    6:'MeanOfTransportation',
    7:'Building',
    8:'NaturalPlace',
    9:'Village',
    10:'Animal',
    11:'Plant',
    12:'Album',
    13:'Film',
    14:'WrittenWork'
}

# Mapping the classes 
df['class_name'] = df['class'].map(class_dict)
df.head()

,class,name,description,class_name
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...,Company
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...,Company
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...,Company
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...,Company
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...,Company


In [7]:
df

,class,name,description,class_name
0,1,E. D. Abbott Ltd,Abbott of Farnham E D Abbott Limited was a Br...,Company
1,1,Schwan-Stabilo,Schwan-STABILO is a German maker of pens for ...,Company
2,1,Q-workshop,Q-workshop is a Polish company located in Poz...,Company
3,1,Marvell Software Solutions Israel,Marvell Software Solutions Israel known as RA...,Company
4,1,Bergan Mercy Medical Center,Bergan Mercy Medical Center is a hospital loc...,Company
...,...,...,...,...
559995,14,Barking in Essex,Barking in Essex is a Black comedy play direc...,WrittenWork
559996,14,Science & Spirit,Science & Spirit is a discontinued American b...,WrittenWork
559997,14,The Blithedale Romance,The Blithedale Romance (1852) is Nathaniel Ha...,WrittenWork
559998,14,Razadarit Ayedawbon,Razadarit Ayedawbon (Burmese: ရာဇာဓိရာဇ် အရေး...,WrittenWork


In [8]:
df['class_name'].value_counts()

NaturalPlace              40000
Artist                    40000
MeanOfTransportation      40000
Building                  40000
WrittenWork               40000
Village                   40000
Company                   40000
OfficeHolder              40000
Album                     40000
Plant                     40000
Film                      40000
Animal                    40000
EducationalInstitution    40000
Athlete                   40000
Name: class_name, dtype: int64

In [19]:
# Lets do some cleaning of this text

"""
      The normal KD(NFKD) will apply the compatibility decomposition i.e. replace all
      compatibility characters with their equivalents. 
      Normal Form KC (NFKC) 1st applies compatibility decomposition, followed by canonical 
      decomposition
"""
    
def clean_it(text,normalize=True):
    # Replacing possible issues with data. We can add or reduce the replacemtent in this chain
    s = str(text).replace(',',' ').replace('"','').replace('\'',' \' ').replace('.',' . ').replace('(',' ( ').\
            replace(')',' ) ').replace('!',' ! ').replace('?',' ? ').replace(':',' ').replace(';',' ').lower()
    
    # normalizing / encoding the text
    if normalize:
   
          s = s.normalize('NFKD').str.encode('ascii','ignore').str.decode('utf-8')
    
    return s

# Now lets define a small function where we can use above cleaning on datasets
def clean_df(data, cleanit= False, shuffleit=False, encodeit=False, label_prefix='__class__'):
    # Defining the new data
    df = data[['name','description']].copy(deep=True)
    df['class'] = label_prefix + data['class'].astype(str) + ' '
    
    # cleaning it
    if cleanit:
        df['name'] = df['name'].apply(lambda x: clean_it(x,encodeit))
        df['description'] = df['description'].apply(lambda x: clean_it(x,encodeit))
    
    # shuffling it
    if shuffleit:
        df.sample(frac=1).reset_index(drop=True)
            
    return df



      
      


In [20]:
%%time

# Transform the datasets using the above clean functions
df_train_cleaned = clean_df(df, True, True)
df_test_cleaned = clean_df(df_test, True, True)

CPU times: user 6.1 s, sys: 377 ms, total: 6.48 s
Wall time: 6.49 s


In [21]:
# Write files to disk as fastText Classifier API reads files from disk

train_file = data_path + '/dbpedia_train.csv'
df_train_cleaned.to_csv(train_file, header = None, index = False, 
                        columns = ['class', 'name', 'description'])

test_file = data_path + '/dbpedia_test.csv'
df_test_cleaned.to_csv(test_file, header=None, index=False, 
                       columns = ['class', 'name', 'description'])


Now that we have the train and test files written into disk in a former fastText wants, we are ready to use it for text classification !

In [22]:
%%time

## using fastText for feature extraction and training
from fasttext import train_supervised
"""
fastText inputs a training file(csv), a model name as input arguments, 
label_prefix refers to the prefix before label string in the dataset.

default is __label__. 
In our dataset, it is __class__.

There are several other parameters which can be seen in :
https://pypi.org/project/fasttext/
"""

model = train_supervised(input=train_file, label="__class__", lr=1.0, epoch=75,
                         loss='ova', wordNgrams = 2, dim=200, thread=2, verbose=100)

CPU times: user 1h 26min 8s, sys: 21.7 s, total: 1h 26min 30s
Wall time: 44min 12s


In [26]:
for k in range(1, 6):

  results = model.test(test_file, k=k)
  print(f"Test Samples: {results[0]} Precision@{k} : {results[1]*100:2.4f} Recall@{k} : {results[2]}*100:2.4f")


Test Samples: 70000 Precision@1 : 87.6114 Recall@1 : 0.8761142857142857*100:2.4f
Test Samples: 70000 Precision@2 : 46.9550 Recall@2 : 0.9391*100:2.4f
Test Samples: 70000 Precision@3 : 31.4710 Recall@3 : 0.9441285714285714*100:2.4f
Test Samples: 70000 Precision@4 : 23.8364 Recall@4 : 0.9534571428571429*100:2.4f
Test Samples: 70000 Precision@5 : 19.1189 Recall@5 : 0.9559428571428571*100:2.4f
